# WTMD χN Reweighting Analysis

Compute free energy and probability distribution at different χN values
using histogram reweighting from WTMD simulation data.

**Usage:**
1. Modify `chin` (simulation χN) and `data_dir` below
2. Adjust `dchiN` to compute F(Ψ) at different χN values
3. Run all cells

**Reference:**
T. M. Beardsley and M. W. Matsen, J. Chem. Phys. 157, 114902 (2022).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy.interpolate import interp1d
from scipy.signal import find_peaks

In [ ]:
# Parameters - Modify these
chin = 13.15                    # Simulation χN
langevin_step = 10_000_000      # Langevin step for statistics file
data_dir = f"lam_{chin:05.2f}/data_simulation"

# χN shift for reweighting (set to 0 for no shift)
dchiN = -0.014

# Renormalization factor (discrete chain model)
z_inf = 0.7691138

print(f"Simulation χN: {chin}")
print(f"Target χN: {chin + dchiN}")

In [ ]:
# Load WTMD statistics
filename = f"{data_dir}/wtmd_statistics_{langevin_step:06d}.mat"
data = loadmat(filename, squeeze_me=True)

psi_range = data['psi_range']
u = data['u']
I0 = data['I0']
dH_psi_A_B = data['dH_psi_A_B']
dT = data['dT']
dpsi = data['dpsi']

# Compute coefficient and bare χN values
coeff = 1/dT + 1
bare_dchiN = dchiN / z_inf
bare_chiN = chin / z_inf

# Free energy from bias potential
F = -u * coeff

In [ ]:
# Plot F(Ψ) - Free energy at simulation χN
plt.figure(figsize=(8, 5))
plt.plot(psi_range, F)
plt.xlabel('Ψ')
plt.ylabel('F(Ψ)')
plt.title(f'Free Energy at χN = {chin}')
plt.grid(True)
plt.show()

In [ ]:
# Plot dF/dχN with threshold filtering
plt.figure(figsize=(8, 5))
I0_norm = I0 / np.max(I0)
mask = I0_norm > 1e-1
x = psi_range[mask]
y = dH_psi_A_B[mask]

plt.plot(x, y)
plt.xlabel('Ψ')
plt.ylabel('∂F/∂χ_AB N')
plt.title('Free Energy Derivative (filtered)')
plt.grid(True)
plt.show()

In [ ]:
# Interpolate dH/dχN to full Ψ range
plt.figure(figsize=(8, 5))
interp_func = interp1d(x, y, kind='linear', fill_value='extrapolate')
y_interp = interp_func(psi_range)

plt.plot(psi_range, y_interp)
plt.xlabel('Ψ')
plt.ylabel('∂F/∂χ_AB N (interpolated)')
plt.title('Interpolated Free Energy Derivative')
plt.grid(True)
plt.show()

In [ ]:
# Compute reweighted free energy at target χN
F_reweighted = F + y_interp * bare_dchiN

plt.figure(figsize=(8, 5))
plt.plot(psi_range, F_reweighted)
plt.xlabel('Ψ')
plt.ylabel('F(Ψ)')
plt.title(f'Reweighted Free Energy at χN = {chin + dchiN:.3f}')
plt.grid(True)
plt.show()

In [ ]:
# Compute probability distribution P(Ψ)
exp_F = np.exp(-F_reweighted)
P_psi = exp_F / np.sum(exp_F) / dpsi

# Find peaks to identify phase coexistence
peaks, _ = find_peaks(-P_psi, prominence=0.01)

if len(peaks) > 0:
    area1 = np.sum(P_psi[:peaks[0]])
    area2 = np.sum(P_psi[peaks[0]:])
    total_area = area1 + area2
    print(f"Phase areas: [{area1:.4f}, {area2:.4f}]")
    print(f"Ordered phase fraction: {area2 / total_area:.4f}")
else:
    print("No significant phase boundary found.")

plt.figure(figsize=(8, 5))
plt.plot(psi_range, P_psi)
plt.xlabel('Ψ')
plt.ylabel('P(Ψ)')
plt.title(f'Probability Distribution at χN = {chin + dchiN:.3f}')
plt.grid(True)
plt.show()